In [61]:
from selenium import webdriver
from selenium.common.exceptions import *
from selenium.webdriver.common.keys import Keys

from datetime import datetime, timedelta
import time

In [74]:
driver = webdriver.Chrome(executable_path = "../driver/chromedriver")
driver.get("http://www.aduanet.gob.pe/aduanas/informao/HRMCFLlega.htm")

# Find manifests

## Input

In [3]:
date_input = driver.find_elements_by_name("CMc2_Fecha1")[0]

In [4]:
last_month = datetime.strftime(datetime.today() - timedelta(25), "%d/%m/%Y")

In [5]:
date_input.clear()
date_input.send_keys(last_month)

In [6]:
button = driver.find_elements_by_css_selector("input[type=button]")[0]

In [7]:
button.click()

## Output

In [8]:
table = driver.find_elements_by_tag_name("table")[3]

In [9]:
table_manifests = table.find_elements_by_css_selector("tr:not(:first-child) a")

In [10]:
manifests = []
for table_manifest in table_manifests:
    split = table_manifest.text.split(" - ")
    manifests.append({
        "year": "20" + split[0],
        "manifest": split[1]
    })

In [ ]:
import json
with open("../db/manifests-20190319.json", "w") as file:
    json.dump(manifests, file)

# Find knowledges

## Input

In [4]:
with open("../db/manifests-20190318.json", "r") as file:
    manifests = json.load(file)

In [5]:
manifests

[{'manifest': '46', 'year': '2019'},
 {'manifest': '383', 'year': '2019'},
 {'manifest': '436', 'year': '2019'},
 {'manifest': '465', 'year': '2019'},
 {'manifest': '494', 'year': '2019'},
 {'manifest': '439', 'year': '2019'},
 {'manifest': '450', 'year': '2019'},
 {'manifest': '453', 'year': '2019'},
 {'manifest': '467', 'year': '2019'},
 {'manifest': '456', 'year': '2019'},
 {'manifest': '462', 'year': '2019'},
 {'manifest': '400', 'year': '2019'},
 {'manifest': '449', 'year': '2019'},
 {'manifest': '458', 'year': '2019'},
 {'manifest': '445', 'year': '2019'},
 {'manifest': '305', 'year': '2019'},
 {'manifest': '505', 'year': '2019'},
 {'manifest': '473', 'year': '2019'},
 {'manifest': '420', 'year': '2019'},
 {'manifest': '396', 'year': '2019'},
 {'manifest': '492', 'year': '2019'},
 {'manifest': '463', 'year': '2019'},
 {'manifest': '464', 'year': '2019'},
 {'manifest': '306', 'year': '2019'},
 {'manifest': '498', 'year': '2019'},
 {'manifest': '440', 'year': '2019'},
 {'manifest':

In [43]:
len(manifests)

238

In [81]:
manifest = manifests[6]
print(manifest)

{'manifest': '450', 'year': '2019'}


In [83]:
driver.get("http://www.aduanet.gob.pe/cl-ad-itconsmanifiesto/manifiestoITS01Alias?accion=cargarFrmConsultaManifiesto&tipo=M")
# Sleep
time.sleep(5)
# Write year input
year_input = driver.find_elements_by_name("CMc1_Anno")[0]
year_input.clear()
year_input.send_keys(manifest["year"])
# Write manifest input
manifest_input = driver.find_elements_by_name("CMc1_Numero")[0]
manifest_input.clear()
manifest_input.send_keys(manifest["manifest"])
# Sleep
time.sleep(2)
# Click
button = driver.find_elements_by_css_selector("input[type=button]")[0]
button.click()
# Sleep
time.sleep(5)
# Get table
tables = driver.find_elements_by_tag_name("table")
try:
    table = tables[3]
    rows = table.find_elements_by_css_selector("tr:not(:first-child)")
    print(len(rows), "knowledges found")
except IndexError:
    print("no knowledges found")

1892 knowledges found


# Output

In [50]:
knowledges = []
for row in rows:
    # Get cells
    cells = row.find_elements_by_tag_name("td")
    
    # Declare knowledge
    knowledge = {
        "year": manifest["year"],
        "manifest": manifest["manifest"]
    }
    
    # Get knowledge identifier
    knowledge["knowledge"] = cells[2].text.strip()
    
    # Get detail number
    knowledge["detail"] = cells[4].text.strip()
    
    # Get consignee
    knowledge["consignee"] = cells[14].text.strip()
    
    # Get shipper
    knowledge["shipper"] = cells[15].text.strip()
    
    # Save knowledge
    knowledges.append(knowledge)

In [51]:
knowledges

[{'consignee': 'CHAMPINONES ABRANTES S.A.',
  'detail': '1',
  'knowledge': 'HLCURTM181248738',
  'manifest': '439',
  'shipper': 'LEGRO MUSHROOM',
  'year': '2019'},
 {'consignee': 'OPPOSITE SEASON LTDA',
  'detail': '2',
  'knowledge': 'HLCUANR190100537',
  'manifest': '439',
  'shipper': 'ARDO NV',
  'year': '2019'},
 {'consignee': 'FRIGORIFICO DE OSORNO',
  'detail': '3',
  'knowledge': 'HLCUANR190102587',
  'manifest': '439',
  'shipper': 'AJC INTERNATIONAL GLOBAL GMBH',
  'year': '2019'},
 {'consignee': 'IMPORTADORA Y COMERCIALIZADORA',
  'detail': '4',
  'knowledge': 'HLCUANR190106142',
  'manifest': '439',
  'shipper': 'AGRISTO NV',
  'year': '2019'},
 {'consignee': 'HIPERMERCADOS TOTTUS S.A',
  'detail': '5',
  'knowledge': 'HLCUANR190108210',
  'manifest': '439',
  'shipper': 'GREENYARD FROZEN BELGIUM',
  'year': '2019'},
 {'consignee': 'WATTSS S.A.',
  'detail': '6',
  'knowledge': 'HLCUANR190119907',
  'manifest': '439',
  'shipper': 'ECOFROST SA',
  'year': '2019'},
 {'con

In [52]:
link = ("http://www.aduanet.gob.pe/cl-ad-itconsmanifiesto/manifiestoITS01Alias?accion=consultar"
        "DetalleConocimientoEmbarque&CMc2_Anno={year}&CMc2_Numero={manifest}&CMc2_NumDet={detail}&"
        "CG_cadu=118&CMc2_TipM=mc&CMc2_numcon={knowledge}").format(**knowledges[1])
print(link)
#print(knowledges[1]["link"])
# driver.get(link)

http://www.aduanet.gob.pe/cl-ad-itconsmanifiesto/manifiestoITS01Alias?accion=consultarDetalleConocimientoEmbarque&CMc2_Anno=2019&CMc2_Numero=439&CMc2_NumDet=2&CG_cadu=118&CMc2_TipM=mc&CMc2_numcon=HLCUANR190100537


In [53]:
len(knowledges)

986

In [56]:
knowledges[36]

{'consignee': 'SOCIEDAD FRUTOS DE LONGUEN LTDA',
 'detail': '37',
 'knowledge': 'HLCURTM181236188',
 'manifest': '439',
 'shipper': 'CNC GRONDSTOFFEN BV',
 'year': '2019'}

# Find containers

## Input

In [59]:
last_index = 35

In [76]:
for index, knowledge in enumerate(knowledges):
    if index <= last_index:
        continue

    # Go to knowledge link
    link = ("http://www.aduanet.gob.pe/cl-ad-itconsmanifiesto/manifiestoITS01Alias?accion=consultar"
            "DetalleConocimientoEmbarque&CMc2_Anno={year}&CMc2_Numero={manifest}&CMc2_NumDet={detail}&"
            "CG_cadu=118&CMc2_TipM=mc&CMc2_numcon={knowledge}").format(**knowledge)
    try:
        driver.get(link)
    except TimeoutException:
        print("knowledge", index, knowledge["knowledge"], "- timeout before extracting")
        break
    time.sleep(5)

    # Get titles
    titles = driver.find_elements_by_css_selector("body > font > b")

    if (len(titles) > 0) and (titles[0].text == "Contenedores"):
        # Get table
        try:
            table = driver.find_elements_by_tag_name("table")[3]
        except IndexError:
            print("knowledge", index, knowledge["knowledge"], "- table not found")
            continue

        # Get rows
        rows = table.find_elements_by_css_selector("tr:not(:first-child)")

        # Get containers
        containers = []
        for row in rows:
            # Get cells
            cells = row.find_elements_by_tag_name("td")

            # Save container number
            try:
                container = cells[0].text.strip()
            except IndexError:
                print("knowledge", index, knowledge["knowledge"], "- container not found")
                continue

            # print("container:", container)
            containers.append(container)

        # Save all containers
        knowledge["containers"] = containers
        print("knowledge", index, knowledge["knowledge"], "- containers saved")

    else:
        print("knowledge", index, knowledge["knowledge"], "- no containers")
    
    last_index = index
    
    time.sleep(5)
    try:
        driver.back()
    except TimeoutException:
        print("knowledge", index, knowledge["knowledge"], "- timeout after extracting")
        break
    time.sleep(5)

knowledge 778 MEDURT462328 - containers saved
knowledge 779 MEDURT463979 - containers saved
knowledge 780 MEDURT464159 - containers saved
knowledge 781 MEDURT471246 - containers saved
knowledge 782 MEDURT473655 - containers saved
knowledge 783 MEDURT473994 - containers saved
knowledge 784 MEDURT475882 - containers saved
knowledge 785 MEDUSD014412 - containers saved
knowledge 786 MEDUTE089023 - containers saved
knowledge 787 MEDUUU756216 - containers saved
knowledge 788 MEDUUU758477 - containers saved
knowledge 789 MEDUUU763279 - containers saved
knowledge 790 MEDUUU768328 - containers saved
knowledge 791 MEDUUU769763 - containers saved
knowledge 792 MEDUUU789076 - containers saved
knowledge 793 MEDUVI008859 - containers saved
knowledge 794 MEDUVI011010 - containers saved
knowledge 795 MEDUVI016076 - containers saved
knowledge 796 MEDUVI016183 - containers saved
knowledge 797 MEDUVI016654 - containers saved
knowledge 798 MEDUVI018171 - containers saved
knowledge 799 MEDUVL662412 - conta

knowledge 956 TL/HBL/037809 - containers saved
knowledge 957 TL/HBL/038110 - containers saved
knowledge 958 TL/HBL/038113 - containers saved
knowledge 959 TL/HBL/038111 - containers saved
knowledge 960 TL/HBL/038109 - containers saved
knowledge 961 TL/HBL/038106 - containers saved
knowledge 962 TL/HBL/038105 - containers saved
knowledge 963 TL/HBL/038165 - containers saved
knowledge 964 TL/HBL/038164 - containers saved
knowledge 965 STSBF3481 - containers saved
knowledge 966 BRASP190103 - containers saved
knowledge 967 HLCUBO2190200490 - containers saved
knowledge 968 HLCUBO2190211010 - containers saved
knowledge 969 HLCUBO2190211540 - containers saved
knowledge 970 HLCUBO2190211551 - containers saved
knowledge 971 HLCUBO2190212459 - containers saved
knowledge 972 HLCUBO2190212460 - containers saved
knowledge 973 HLCUBO2190204213 - containers saved
knowledge 974 HLCUBO2190211975 - containers saved
knowledge 975 BOG516433 - containers saved
knowledge 976 MDE844687 - containers saved
kno

In [79]:
filename = "../db/manifests/{year}-{manifest}.json".format(**manifest)
with open(filename, "w") as file:
    json.dump(knowledges, file)

'\nwww.aduanet.gob.pe/cl-ad-itconsmanifiesto/manifiestoITS01Alias?accion=consultarDetalleConocimientoEmbarque\n&CMc2_Anno={year}&CMc2_Numero={manifest}&CMc2_NumDet=1&CG_cadu=118&CMc2_TipM=mc&CMc2_numcon={knowledge}\n'

# End

In [84]:
driver.close()

Dentro de un contenedor
- cada cliente llena cierto volumen que se coloca en un contenedor
- un agente de carga origen envía a un destino => para aduanas se maneja con los manifiestos

cuando la nave llega al puerto

ABCDE
AB: dos primeras iniciales del pais
CDE: tres primeras iniciales del puerto

DFSU2255679

table[3]

tr:not(:first-child)

td[0]
